In [1]:
import os 

In [2]:
%pwd

'/home/devil/Text-Summerization-/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/devil/Text-Summerization-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from TextSummerizer.constants import *
from TextSummerizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os 
import urllib.request as request
import zipfile
from TextSummerizer.logging import logger
from TextSummerizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )
                logger.info(f"{filename} download! with following info: \n{headers}")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
        except Exception as e:
            logger.error(f"Error downloading the file: {e}")
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # Assuming download_file and extract_zip_file can raise specific exceptions
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except FileNotFoundError as file_not_found_error:
    print(f"File not found: {file_not_found_error}")

except Exception as e:
    logger.error(f"An unexpected error occurred: {e}")
    raise


[2023-12-16 16:18:29,050: INFO: common: YAML file 'config/config.yaml' loaded successfully]
[2023-12-16 16:18:29,056: INFO: common: YAML file 'params.yaml' loaded successfully]
[2023-12-16 16:18:29,059: INFO: common: Created directory at: artifacts]
[2023-12-16 16:18:29,062: INFO: common: Created directory at: artifacts/data_ingestion]
[2023-12-16 16:18:31,532: INFO: 4129404692: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 2064395
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9ea01f63642340ccfa34b0cc9e4823bf0db967a0054326ffae19e3520d06085e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8D04:3CDF11:9A38D:A8488:657D80A1
Accept-Ranges: bytes
Date: Sat, 16 Dec 2023 10:49:06 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4721-BOM
X-Cache: MI